# Scrapping Amazon Reviews

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime

In [2]:
links = pd.read_excel('dataset.xlsx')

In [3]:
links.head(5)

,Book Title,Amazon link,Goodreads link
0,Finding the Mother Tree,https://www.amazon.com/Finding-Mother-Tree-Dis...,https://www.goodreads.com/book/show/54976983-f...
1,Flashlight Night,https://www.amazon.com/Flashlight-Night-Advent...,https://www.goodreads.com/book/show/53370054-f...
2,Treason,https://www.amazon.com/Treason-Stone-Barringto...,https://www.goodreads.com/book/show/45213808-t...
3,The Giver of Stars,https://www.amazon.com/Giver-Stars-Novel-Jojo-...,https://www.goodreads.com/book/show/53258232-t...
4,Resistance Reborn (Star Wars),https://www.amazon.com/Resistance-Reborn-Star-...,https://www.goodreads.com/book/show/45452530-r...


In [4]:
amazon_links = links['Amazon link']

In [5]:
data_asin=[]
titles=[]

In [6]:
for i in range (len(amazon_links)):
    url=amazon_links[i]
    asin=url.split("/dp/")[1]
    title=url.split("/dp/")[0].split('.com/')[1]
    titles.append(title)
    data_asin.append(asin)

In [7]:
base_urls=[]
length = len(amazon_links)

In [8]:
for i in range (length):
    base_url='https://www.amazon.com/'+titles[i]+'/product-reviews/'+data_asin[i]+'ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber='
    base_urls.append(base_url)

In [9]:
def reviews(base_url):
    reviews=[]
    ratings=[]
    review_dates=[]
    HEADERS = ({'User-Agent':
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})
    for i in range (1,11):
        URL = base_url+str(i)
        webpage = requests.get(URL, headers=HEADERS)
        soup = BeautifulSoup(webpage.content, "lxml")
        review = soup.find_all("span", attrs={'data-hook':'review-title'})
        review_date = soup.find_all("span", attrs={'data-hook':'review-date'})
        rating = soup.find_all("span", attrs={'class':'a-icon-alt'})
    
        for j in range(len(review)):
            reviews.append(review[j].text)
        for j in range(len(review_date)):
            review_dates.append(review_date[j].string.strip())
        for j in range(len(rating)):
            ratings.append(rating[j].string.strip())
        
    for j in range(len(reviews)):
        reviews[j]=reviews[j].strip('\n')
    
    for j in range(len(ratings)):
        ratings[j]=ratings[j].split()[0]
    
    for j in range(len(review_dates)):
        review_dates[j]=review_dates[j].split('on')[1].strip()
        review_dates[j]=datetime.strptime(review_dates[j],'%B %d, %Y').strftime('%Y-%m-%d')
        
    df=pd.DataFrame(list(zip(reviews,ratings,review_dates)),columns =['Review', 'Rating','Date'])
    return df

In [10]:
amazon_reviews={}

In [12]:
for i in range (length):
    df = reviews(base_urls[i])
    d={titles[i]:df}
    amazon_reviews.update(d)

In [14]:
reviews=df_piece = pd.concat(amazon_reviews)

In [18]:
reviews['sourec']='Amazon'

In [19]:
reviews.head()

Review  \
Finding-Mother-Tree-Discovering-Wisdom 0  The Mother of all Tree Books, learn about how ...   
                                       1  I found the mother tree, I discovered the Wisd...   
                                       2  The Mother of all Tree Books, learn about how ...   
                                       3  I found the mother tree, I discovered the Wisd...   
                                       4  The Mother of all Tree Books, learn about how ...   

                                         Rating        Date  sourec  
Finding-Mother-Tree-Discovering-Wisdom 0    4.6  2021-05-04  Amazon  
                                       1    5.0  2021-05-04  Amazon  
                                       2    1.0  2021-05-17  Amazon  
                                       3    5.0  2021-05-09  Amazon  
                                       4    5.0  2021-05-05  Amazon

In [20]:
reviews.to_csv('amazon_review.csv',index=False)